<a href="https://colab.research.google.com/github/vinayshanbhag/AustinRadarTraffic/blob/master/Austin_Traffic_Radar_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Austin Radar Traffic Sensor data - Visualizing time series data on a map¶

Data Source: City of Austin https://data.austintexas.gov/Transportation-and-Mobility/Radar-Traffic-Counts/i626-g7ub

In [0]:
import pandas as pd
import folium
from folium import plugins
import requests, json
from datetime import datetime

from google.colab import drive

#### Load traffic volume data

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
f = '/content/drive/My Drive/Radar_Traffic_Counts.csv'

In [0]:
df = pd.read_csv(f,)
df = df[df.Year==2019]

#### Load sensor location data

In [0]:
filter_clause = '$where='
for idx, val in enumerate(df['KITS ID'].unique()):
  if idx == 0: filter_clause = filter_clause + f'kits_id={val}'
  else:filter_clause = filter_clause + f' or kits_id={val}'

In [0]:
sensors = requests.get(f"https://data.austintexas.gov/resource/6yd9-yz29.json?{filter_clause}").json()

In [0]:
sensors = [s for s in sensors if s.get('kits_id','')]

In [0]:
sensors = {int(s['kits_id']):s for s in sensors if int(s['kits_id']) in df['KITS ID'].unique()}

#### Prepare data

<p>
Drop sensor data for any sensors we can't locate.

Add location names and geo codes.

Aggregate traffic volumes by hour of day for each location.
</p>

In [0]:
df = df[df['KITS ID'].isin(sensors.keys())].copy()

In [0]:
df['location_name'] = df['KITS ID'].apply(lambda x: sensors[x]['location_name'])
df['location_latitude'] = df['KITS ID'].apply(lambda x: float(sensors[x]['location_latitude']))
df['location_longitude'] = df['KITS ID'].apply(lambda x: float(sensors[x]['location_longitude']))

In [11]:
hourly_vol = df.groupby(['location_name','location_latitude','location_longitude','Direction','Hour']).agg({'Volume':'sum'}).reset_index()
hourly_vol.sample(2)

,location_name,location_latitude,location_longitude,Direction,Hour,Volume
361,LAMAR BLVD / N LAMAR SB TO W 15TH RAMP,30.279604,-97.750472,SB,1,22367
505,LAMAR BLVD / ZENNIA ST,30.319905,-97.730292,SB,1,27580


#### Construct Time series Geo JSON and map

In [0]:
colors = ['green','yellow','orange','red']
def get_color(f, volume, location):
  q = f[(f['location_name']==location)].Volume.quantile([0.4,0.5,0.75]).values
  if volume < q[0]:
      return colors[0]
  elif volume > q[0] and volume < q[1]:    
    return colors[1]
  elif volume > q[1] and volume < q[2]:    
    return colors[2]
  else:
    return colors[3]

In [0]:
lane_coord = {
  "2021 BLK KINNEY AVE (NW 300ft NW of Lamar)": {
    "None": [[-97.769321, 30.250531],[-97.770732, 30.248283]]
  },
  " CAPITAL OF TEXAS HWY / LAKEWOOD DR": {
    "NB": [[-97.783021, 30.373691],[-97.787913, 30.369489]],
    "SB": [[-97.783289, 30.373951],[-97.788245, 30.369813]]
  },
  "400 BLK AZIE MORTON RD (South of Barton Springs Rd)": {
    "SB": [[-97.765106, 30.264723],[-97.766051, 30.264064]],
    "NB": [[-97.765065, 30.264691],[-97.766010, 30.264034]]
  },
  " BURNET RD / PALM WAY (IBM DRIVEWAY)": {
    "SB": [[-97.717408, 30.402870],[-97.718401, 30.401129]],
    "NB": [[-97.717271, 30.402814],[-97.718264, 30.401063]]
  },
  "1000 BLK W CESAR CHAVEZ ST (H&B Trail Underpass)": {
    "EB": [[-97.761858, 30.269488],[-97.758660, 30.268068]],
    "WB": [[-97.761788, 30.269595],[-97.758603, 30.268185]]
  },
  " LAMAR BLVD / SANDRA MURAIDA WAY (Lamar Bridge)": {
    "NB": [[-97.755788,30.267502],[-97.756794,30.265083]],
    "SB": [[-97.756007,30.267502],[-97.757013,30.265083]]
  },
  "100 BLK S CONGRESS AVE (Congress Bridge)": {
    "NB": [[-97.745429, 30.260732],[-97.746706, 30.257173]],
    "SB": [[-97.745665, 30.260834],[-97.746931, 30.257387]]
  },
  " LAMAR BLVD / N LAMAR SB TO W 15TH RAMP": {
    "NB": [[-97.750308, 30.280748],[-97.750542, 30.278855]],
    "SB": [[-97.750682, 30.280951],[-97.750878, 30.278861]]
  },
  " CONGRESS AVE / JOHANNA ST (Fulmore Middle School)": {
    "SB": [[-97.751431, 30.245143],[-97.752028, 30.243520]],
    "NB": [[-97.751262, 30.245105],[-97.751891, 30.243416]]
  },
  "1612 BLK S LAMAR BLVD (Collier)": {
    "SB": [[-97.764003, 30.252651],[-97.766610, 30.250158]],
    "NB": [[-97.763778, 30.252465],[-97.766374, 30.249945]]
  },
  " LAMAR BLVD / SHOAL CREEK BLVD": {
    "SB":[[-97.748116,30.294795],[-97.747198,30.293276],[-97.747148,30.292940],[-97.747286,30.292614],[-97.747550,30.292354],[-97.747801,30.292245],[-97.749699,30.291876]],
    "NB": [[-97.747972, 30.295011],[-97.747113, 30.293652],[-97.74696, 30.293395],[-97.746905, 30.293168],[-97.746912, 30.292905],[-97.746967, 30.292707],[-97.747071, 30.292492],[-97.747237, 30.292306],[-97.747466, 30.292139],[-97.747688, 30.292043],[-97.749829, 30.29163]]
  },
  "700 BLK E CESAR CHAVEZ ST": {
    "WB": [[-97.738738, 30.261854],[-97.736822, 30.261315]],
    "EB": [[-97.738778, 30.261727],[-97.736862, 30.261177]]
  },
  " LAMAR BLVD / MANCHACA RD": {
    "NB": [[-97.780499, 30.244798],[-97.783220, 30.242842]],
    "SB": [[-97.780651, 30.244934],[-97.783367, 30.242983]]
  },
  " CAPITAL OF TEXAS HWY / WALSH TARLTON LN": {
    "SB": [[-97.816003, 30.262221],[-97.810875, 30.255669]],
    "NB": [[-97.815402, 30.262499],[-97.810349, 30.256031]]
  },
  " LAMAR BLVD / ZENNIA ST": {
    "SB": [[-97.729588, 30.321111],[-97.731580, 30.317937]],
    "NB": [[-97.729272, 30.321058],[-97.731325, 30.317770]]
  },
  " BURNET RD / RUTLAND DR": {
    "NB": [[-97.723120, 30.386833],[-97.724610, 30.380611]],
    "SB": [[-97.723421, 30.386971],[-97.724816, 30.380900]]
  },
  " CAPITAL OF TEXAS HWY / CEDAR ST": {
    "SB": [[-97.802593, 30.341246],[-97.804953, 30.337514]],
    "NB": [[-97.802292, 30.341172],[-97.804738, 30.337357]]
  },
  "3201 BLK S LAMAR BLVD (BROKEN SPOKE)": {
    "None": [[-97.784670, 30.241911],[-97.789262, 30.238682]]
  }
}


In [14]:
# Date slider - We are visualizing volume at each location by hour of day, using several years of data. Date can be ignored.
dt = datetime.today().replace(hour=0, minute=0, second=0, microsecond=0)

# construct geo json
lanes = [
    {
      "type": "Feature",
      "geometry": {
        "type": "LineString",
        "coordinates": lane_coord[row[0]][row[3]],
      },
      "properties": {
        'style':{
          'color': get_color(hourly_vol,row[5],row[0]),
          'weight': 3
        },
        "times": [dt.replace(hour=row[4], second=i).strftime("%Y-%m-%dT%T") for i in range(len(lane_coord[row[0]][row[3]]))]
      }
    }
    for idx, row in enumerate(hourly_vol.values)
  ]

data = {
  "type": "FeatureCollection",
  "features": lanes
}

m = folium.Map([30.266373, -97.756343], zoom_start=14)
m.add_child(plugins.TimestampedGeoJson(data
                                       ,period='PT1H'
                                       ,transition_time=100
                                       ,add_last_point=False)
)
m